In [234]:
import numpy as np
from unittest import mock

from collections import defaultdict
from grid import Grid, RandomGridGenerator

from utils import two_int_to_hash

In [218]:
# GLOBAL CONFIGS
GRID_NUMBER_OF_ROWS = 10
GRID_NUMBER_OF_COLS = 10

PREDATOR_VISION_DIST = 3
PREY_VISION_DIST = 2

PREDATOR_VALUE = 1
PREY_VALUE = -1

PREDATOR_APPROXIMATE_SHARE = 0.4
PREY_APPROXIMATE_SHARE = 0.6

In [210]:
rgg = RandomGridGenerator(GRID_NUMBER_OF_ROWS, GRID_NUMBER_OF_COLS)

generated_grid = rgg.getGrid(round(PREDATOR_APPROXIMATE_SHARE * 100) * [PREDATOR_VALUE] + round(PREY_APPROXIMATE_SHARE * 100) * [PREY_VALUE])
generated_grid


array([[ 0,  0,  1, -1,  0,  0,  0, -1,  0,  0],
       [ 0,  0, -1, -1,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0,  1,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1,  1],
       [ 0,  0,  0,  0,  0,  0,  0, -1,  0,  1],
       [ 0,  0,  0,  0,  1,  0,  1,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  1,  1]])

In [212]:
grid = Grid(generated_grid)

In [222]:
predator_model = mock.Mock()
prey_model = mock.Mock()

In [235]:
class Simulation:


    def get_chosen_dests(self, grid: Grid):
        chosen_dests = defaultdict(lambda: [])
        for row, line in enumerate(grid.grid):
            for col, item in enumerate(line):
                if item == 0:
                    continue

                if item == PREDATOR_VALUE:
                    vision_space = grid.getNeighbors(row, col, PREDATOR_VISION_DIST)
                    dest_row, dest_col = 1,4 # predator_model.make_desiscion(vision_space)

                if item == PREY_VALUE:
                    vision_space = grid.getNeighbors(row, col, PREY_VISION_DIST)
                    dest_row, dest_col = 3, 4 # prey_model.make_desiscion(vision_space)

                chosen_dests[ two_int_to_hash(dest_row, dest_col)].append(item)

        return chosen_dests

In [230]:
grid.grid

array([[ 0,  0,  1, -1,  0,  0,  0, -1,  0,  0],
       [ 0,  0, -1, -1,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0,  1,  0],
       [ 0,  0,  0, -1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0, -1,  1],
       [ 0,  0,  0,  0,  0,  0,  0, -1,  0,  1],
       [ 0,  0,  0,  0,  1,  0,  1,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  1,  1]])

In [237]:
s = Simulation()

s.get_chosen_dests(grid)

defaultdict(<function __main__.Simulation.get_chosen_dests.<locals>.<lambda>()>,
            {31528: [1, 1, 1, 1, 1, 1, 1, 1, 1],
             31602: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]})